In [299]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from ast import literal_eval


In [300]:
var_dict = {
    'T':'Time_Spend_Company',
    'D':'Department',
    'NP':'Number_Projects',
    'P':'Promotion_Last_5_Years',
    'L':'Left',
    'H':'Average_Montly_Hours',
    'WA':'Work_Accident',
    'LE':'Last_Evaluation',
    'S':'Salary',
    'SL':'Satisfaction_Level'
}

def item2dict(item):
    ## splitto la stringa
    item_split = item.split('_')
    # definisco la lista variabile,valore
    item_list = {var_dict[item_split[1]] : item_split[0]}
    return item_list


In [301]:
def df2df_set(df):
    """funzione che converte 
    le stringhe relative alle regole in 
    liste di dizionari """

    df_new = df.copy()

    df_new = df_new.drop(['Antecedent','Consequent'],axis=1)

    Antecedent=[]
    for i,item in enumerate(df.Antecedent):
        Antecedent.append(set(literal_eval(item)))

    df_new=df_new.assign(Antecedent=Antecedent)

    Consequent=[]
    for i,item in enumerate(df.Consequent):
        Consequent.append(item2dict(item))

    df_new=df_new.assign(Consequent=Consequent)
        
    return df_new        


# Analisi regole Per LEFT

In [302]:
df = pd.read_csv("../../data/rules/rules_supp5_conf50.csv")
df.shape
#df_filtered = df.query('Supp>0.2') # perchè non è maggiore di 20???
#df_filtered = df_filtered.query('Lift>1')
df_left = df.query(" Consequent=='Y_L' and Lift>1")

df_left.to_csv("../../code/rules/df_left_all.csv")
df_left.describe()

# converto in dataset di set
df_left_set=df2df_set(df_left)
df_left_set.head()

# aggiungo la lunghezza della parte antecedente
N_items=[]
ant = df_left_set.Antecedent
N_items = []
for i,ant in enumerate(df_left_set.Antecedent):
    N_items.append(len(df_left_set.Antecedent.iloc[i]))


df_left_set['N_items']=N_items
df_left_set.head()

     Supp   Conf   Lift                     Antecedent       Consequent  \
79  0.050  0.774  3.252  {intensive_H, N_P, N_WA, 5_T}  {u'Left': u'Y'}   
82  0.050  0.772  3.243       {intensive_H, N_WA, 5_T}  {u'Left': u'Y'}   
85  0.053  0.741  3.111        {intensive_H, N_P, 5_T}  {u'Left': u'Y'}   
87  0.053  0.737  3.094             {intensive_H, 5_T}  {u'Left': u'Y'}   
90  0.053  0.615  2.583               {N_P, N_WA, 5_T}  {u'Left': u'Y'}   

    N_items  
79        4  
82        3  
85        3  
87        2  
90        3  

# Inizio pruning delle regole

In [303]:
# pre-filtering
df_left_filtered=df_left_set.query('Conf>0.5')
df_left_filtered.describe()

             Supp        Conf        Lift     N_items
count  264.000000  264.000000  264.000000  264.000000
mean     0.070557    0.811826    3.409856    4.121212
std      0.020260    0.131242    0.551197    1.293625
min      0.050000    0.507000    2.131000    1.000000
25%      0.053000    0.720750    3.028000    3.000000
50%      0.061500    0.852500    3.580500    4.000000
75%      0.086000    0.924250    3.882500    5.000000
max      0.154000    0.977000    4.103000    8.000000

In [304]:
# rimuovo quelli le regole contenenti quelli che non hanno avuto incidenti
data= df_left_filtered.copy()

indici_remove=[]
for i in range(data.shape[0]):
    if set(['N_WA']).issubset( data.iloc[i].Antecedent ):
        indici_remove.append(i)
#        print i
        #         print data.iloc[i].Antecedent
print len(indici_remove)
data.drop(data.index[indici_remove],inplace=True)
data.describe()
data_clean=data
data_clean.head()
del(data)

136


In [312]:
data=data_clean.query('Conf>0.9')



data=data.sort_values(by='N_items')
data_pruned=data.copy()

#print data_pruned.head()
# ordino

# prendo la prima
row=0
while row<data_pruned.shape[0] :
    rule = data.iloc[row].Antecedent
    supp_rule = data.iloc[row].Supp
    
    #print row
    indici_remove=[]
    for i in range((row+1),(data_pruned.shape[0])):
        successiva = data_pruned.iloc[i]
        if rule.issubset(successiva.Antecedent) and supp_rule==successiva.Supp:
            indici_remove.append(i)
            #print supp_rule
            #print successiva.Supp
    data_pruned.drop(data_pruned.index[[indici_remove]],inplace=True)
    del(indici_remove)
    
    row+=1



In [313]:
data_pruned.describe()
#data_pruned

            Supp       Conf       Lift    N_items
count  14.000000  14.000000  14.000000  14.000000
mean    0.071714   0.926143   3.889500   4.142857
std     0.018235   0.013581   0.057535   0.770329
min     0.053000   0.905000   3.800000   3.000000
25%     0.055000   0.918250   3.855750   4.000000
50%     0.062000   0.925500   3.885500   4.000000
75%     0.086000   0.935000   3.927500   4.000000
max     0.101000   0.952000   3.999000   6.000000

In [263]:
df_left[data_]
data_pruned.index
#data_pruned.to_csv("../../data/best_rules.csv")

Int64Index([1351, 1095, 2817, 1255, 1235, 1371, 1963,  185, 2673, 2797, 2793,
            2629,  181,  145, 2697, 2669, 2624, 2692],
           dtype='int64')

In [274]:
data_best_rules=df_left.loc[data_pruned.index]
data_pruned.to_csv("../../data/rules/best_rules.csv")





In [326]:
def prune_rules(data):

    data=data.sort_values(by='N_items')
    data_pruned=data.copy()

    #print data_pruned.head()
    # ordino

    # prendo la prima
    row=0
    while row<data_pruned.shape[0] :
        rule = data.iloc[row].Antecedent
        supp_rule = data.iloc[row].Supp

        #print row
        indici_remove=[]
        for i in range((row+1),(data_pruned.shape[0])):
            successiva = data_pruned.iloc[i]
            if rule.issubset(successiva.Antecedent) and (supp_rule==successiva.Supp):
                indici_remove.append(i)
                #print supp_rule
                #print successiva.Supp
        data_pruned.drop(data_pruned.index[[indici_remove]],inplace=True)
        del(indici_remove)

        row+=1
    return data_pruned



In [346]:
data=data_clean.query('Conf>0.9')
data.sort_values(by='N_items').describe()
data_pruned = prune_rules(data)
data_pruned.describe()


            Supp       Conf       Lift    N_items
count  14.000000  14.000000  14.000000  14.000000
mean    0.071714   0.926143   3.889500   4.142857
std     0.018235   0.013581   0.057535   0.770329
min     0.053000   0.905000   3.800000   3.000000
25%     0.055000   0.918250   3.855750   4.000000
50%     0.062000   0.925500   3.885500   4.000000
75%     0.086000   0.935000   3.927500   4.000000
max     0.101000   0.952000   3.999000   6.000000

In [347]:
data_pruned.to_csv("../../data/rules/pruned_rules.csv")